# ValidMind for model validation 3 — Developing a potential challenger model

Learn how to use ValidMind for your end-to-end model validation process with our series of four introductory notebooks. In this third notebook, develop a potential challenger model and then pass your model and its predictions to ValidMind.

A *challenger model* is an alternate model that attempt to outperform the champion model, ensuring that the best performing fit-for-purpose model is always considered for deployment. Challenger models also help avoid over-reliance on a single model, and allow testing of new features, algorithms, or data sources without disrupting the production lifecycle.

## Prerequisites

In order to develop potential challenger models with this notebook, you'll need to first have:

- [x] Registered a model within the ValidMind Platform and granted yourself access to the model as a validator
- [x] Installed the ValidMind Library in your local environment, allowing you to access all its features
- [x] Learned how to import and initialize datasets for use with ValidMind
- [x] Understood the basics of how to run and log tests with ValidMind

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>Need help with the above steps?</b></span>
<br></br>
Refer to the first two notebooks in this series:

- <a href="1-set_up_validmind_for_validation.ipynb" style="color: #DE257E;"><b>1 — Set up the ValidMind Library for validation</b></a>
- <a href="2-start_validation_process.ipynb" style="color: #DE257E;"><b>2 — Start the model validation process</b></a>

</div>



## Setting up

This section should be quite familiar to you — as we performed the same actions in the previous notebook, **[2 — Start the model validation process](2-start_validation_process.ipynb)**.

### Initialize the ValidMind Library

As usual, let's first connect up the ValidMind Library to our model we previously registered in the ValidMind Platform:

1. In a browser, [log in to ValidMind](https://docs.validmind.ai/guide/configuration/log-in-to-validmind.html).

2. In the left sidebar, navigate to **Inventory** and select the model you registered for this "ValidMind for model validation" series of notebooks.

3. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, [load your model identifier credentials from an `.env` file](https://docs.validmind.ai/developer/model-documentation/store-credentials-in-env-file.html) or replace the placeholder with your own code snippet:

In [ ]:
# Make sure the ValidMind Library is installed

%pip install -q validmind

# Load your model identifier credentials from an `.env` file

%load_ext dotenv
%dotenv .env

# Or replace with your code snippet

import validmind as vm

vm.init(
    # api_host="...",
    # api_key="...",
    # api_secret="...",
    # model="...",
)

### Import the sample dataset

Next, we'll load in the sample [Bank Customer Churn Prediction](https://www.kaggle.com/datasets/shantanudhakadd/bank-customer-churn-prediction) dataset used to develop the champion model that we will independently preprocess:

In [ ]:
# Load the sample dataset
from validmind.datasets.classification import customer_churn as demo_dataset

print(
    f"Loaded demo dataset with: \n\n\t• Target column: '{demo_dataset.target_column}' \n\t• Class labels: {demo_dataset.class_labels}"
)

raw_df = demo_dataset.load_data()

#### Preprocess the dataset

We’ll apply a simple rebalancing technique to the dataset before continuing:

In [ ]:
import pandas as pd

raw_copy_df = raw_df.sample(frac=1)  # Create a copy of the raw dataset

# Create a balanced dataset with the same number of exited and not exited customers
exited_df = raw_copy_df.loc[raw_copy_df["Exited"] == 1]
not_exited_df = raw_copy_df.loc[raw_copy_df["Exited"] == 0].sample(n=exited_df.shape[0])

balanced_raw_df = pd.concat([exited_df, not_exited_df])
balanced_raw_df = balanced_raw_df.sample(frac=1, random_state=42)

Let’s also quickly remove highly correlated features from the dataset using the output from a ValidMind test.

As you know, before we can run tests you’ll need to initialize a ValidMind dataset object with the [`init_dataset` function](https://docs.validmind.ai/validmind/validmind.html#init_dataset):

In [ ]:
# Register new data and now 'balanced_raw_dataset' is the new dataset object of interest
vm_balanced_raw_dataset = vm.init_dataset(
    dataset=balanced_raw_df,
    input_id="balanced_raw_dataset",
    target_column="Exited",
)

With our balanced dataset initialized, we can then run our test and utilize the output to help us identify the features we want to remove:

In [ ]:
# Run HighPearsonCorrelation test with our balanced dataset as input and return a result object
corr_result = vm.tests.run_test(
    test_id="validmind.data_validation.HighPearsonCorrelation",
    params={"max_threshold": 0.3},
    inputs={"dataset": vm_balanced_raw_dataset},
)

In [ ]:
# From result object, extract table from `corr_result.tables`
features_df = corr_result.tables[0].data
features_df

In [ ]:
# Extract list of features that failed the test
high_correlation_features = features_df[features_df["Pass/Fail"] == "Fail"]["Columns"].tolist()
high_correlation_features

In [ ]:
# Extract feature names from the list of strings
high_correlation_features = [feature.split(",")[0].strip("()") for feature in high_correlation_features]
high_correlation_features

We can then re-initialize the dataset with a different `input_id` and the highly correlated features removed and re-run the test for confirmation:

In [ ]:
# Remove the highly correlated features from the dataset
balanced_raw_no_age_df = balanced_raw_df.drop(columns=high_correlation_features)

# Re-initialize the dataset object
vm_raw_dataset_preprocessed = vm.init_dataset(
    dataset=balanced_raw_no_age_df,
    input_id="raw_dataset_preprocessed",
    target_column="Exited",
)

In [ ]:
# Re-run the test with the reduced feature set
corr_result = vm.tests.run_test(
    test_id="validmind.data_validation.HighPearsonCorrelation",
    params={"max_threshold": 0.3},
    inputs={"dataset": vm_raw_dataset_preprocessed},
)

## Import the champion model

With our raw dataset assessed and preprocessed, let's go ahead and import the champion model submitted by the model development team in the format of a `.pkl` file: **[lr_model_champion.pkl](lr_model_champion.pkl)**

In [ ]:
# Import the champion model
import pickle as pkl

with open("lr_model_champion.pkl", "rb") as f:
    log_reg = pkl.load(f)

## Split the preprocessed dataset

With our dummy model imported, raw dataset rebalanced with highly correlated features removed, let's now **spilt our dataset into train and test** in preparation for training our potential challenger model.

To start, let's grab the first few rows from the `balanced_raw_no_age_df` dataset we initialized earlier:

In [ ]:
balanced_raw_no_age_df.head()

Before training the model, we need to encode the categorical features in the dataset:

- Use the `OneHotEncoder` class from the `sklearn.preprocessing` module to encode the categorical features.
- The categorical features in the dataset are `Geography` and `Gender`.

In [ ]:
balanced_raw_no_age_df = pd.get_dummies(
    balanced_raw_no_age_df, columns=["Geography", "Gender"], drop_first=True
)
balanced_raw_no_age_df.head()

Splitting our dataset into training and testing is essential for proper validation testing, as this helps assess how well the model generalizes to unseen data:

- We begin by dividing our preprocessed dataset into feature variables (`X`) and the target variable (`y`), which indicates whether a customer exited.
- Using `train_test_split`, we randomly allocate 80% of the data to the training set (`X_train`, `y_train`) and 20% to the test set (`X_test`, `y_test`), ensuring both sets are representative of the full distribution.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the input and target variables
X = balanced_raw_no_age_df.drop("Exited", axis=1)
y = balanced_raw_no_age_df["Exited"]
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

### Initialize the split datasets

Next, let's initialize the training and testing datasets so they are available for use:

In [ ]:
train_df = X_train
train_df["Exited"] = y_train
test_df = X_test
test_df["Exited"] = y_test

vm_train_ds = vm.init_dataset(
    input_id="train_dataset_final",
    dataset=train_df,
    target_column="Exited",
)

vm_test_ds = vm.init_dataset(
    input_id="test_dataset_final",
    dataset=test_df,
    target_column="Exited",
)

## Training a potential challenger model

We're curious how an alternate model compare to our champion model, so let's train a challenger random forest classification model as a basis for our testing.

Our selected option below offers decreased complexity in terms of implementation — such as lessened manual preprocessing — which can reduce the amount of risk for implementation. However, model risk is not calculated in isolation from a single factor, but rather in consideration with trade-offs in predictive performance, ease of interpretability, and overall alignment with business objectives.

### Random forest classification model

A *random forest classification model* is an ensemble machine learning algorithm that uses multiple decision trees to classify data. In ensemble learning, multiple models are combined to improve prediction accuracy and robustness.

Random forest classification models generally have higher accuracy because they capture complex, non-linear relationships, but as a result they lack transparency in their predictions.

In [ ]:
# Import the Random Forest Classification model
from sklearn.ensemble import RandomForestClassifier

# Create the model instance with 50 decision trees
rf_model = RandomForestClassifier(
    n_estimators=50,
    random_state=42,
)

# Train the model
rf_model.fit(X_train, y_train)

### Extract predicted probabilities

With our challenger model trained, let's extract the predicted probabilities from our two models:

In [ ]:
# Champion — Logistic regression model
# train_log_prob = log_reg.predict_proba(X_train)[:, 1]
# test_log_prob = log_reg.predict_proba(X_test)[:, 1]

# Challenger — Random forest classification model
# train_rf_prob = rf_model.predict_proba(X_train)[:, 1]
# test_rf_prob = rf_model.predict_proba(X_test)[:, 1]

#### Compute binary predictions

Next, we'll convert the probability predictions from our three models into a binary, based on a threshold of `0.3`:

If the probability is greater than `0.3`, the prediction becomes `1` (positive).
Otherwise, it becomes `0` (negative).

In [ ]:

# cut_off_threshold = 0.3

# Champion — Logistic regression model
# train_log_binary_predictions = (train_log_prob > cut_off_threshold).astype(int)
# test_log_binary_predictions = (test_log_prob > cut_off_threshold).astype(int)

# Challenger — Random forest classification model
# train_rf_binary_predictions = (train_rf_prob > cut_off_threshold).astype(int)
# test_rf_binary_predictions = (test_rf_prob > cut_off_threshold).astype(int)

## Initializing the model objects

### Initialize the model objects

In addition to the initialized datasets, you'll also need to initialize a ValidMind model object (`vm_model`) that can be passed to other functions for analysis and tests on the data for each of our two models.

You simply initialize this model object with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model):

In [ ]:
# Initialize the champion logistic regression model
vm_log_model = vm.init_model(
    log_reg,
    input_id="log_model_champion",
)

# Initialize the challenger random forest classification model
vm_rf_model = vm.init_model(
    rf_model,
    input_id="rf_model",
)

### Assign predictions*

With our models registered, we'll move on to assigning both the predictive probabilities coming directly from each model's predictions, and the binary prediction after applying the cutoff threshold described in the Compute binary predictions step above.

- The [`assign_predictions()` method](https://docs.validmind.ai/validmind/validmind/vm_models.html#VMDataset.assign_predictions) from the `Dataset` object can link existing predictions to any number of models.
- This method links the model's class prediction values and probabilities to our `vm_train_ds` and `vm_test_ds` datasets.

In [ ]:
# Champion — Logistic regression model
# vm_train_ds.assign_predictions(
#     model=vm_log_model,
#     prediction_values=train_log_binary_predictions,
#     prediction_probabilities=train_log_prob,
# )

# vm_test_ds.assign_predictions(
#     model=vm_log_model,
#     prediction_values=test_log_binary_predictions,
#     prediction_probabilities=test_log_prob,
# )

# Challenger — Random forest classification model
# vm_train_ds.assign_predictions(
#     model=vm_rf_model,
#     prediction_values=train_rf_binary_predictions,
#     prediction_probabilities=train_rf_prob,
# )

# vm_test_ds.assign_predictions(
#     model=vm_rf_model,
#     prediction_values=test_rf_binary_predictions,
#     prediction_probabilities=test_rf_prob,
# )

If no prediction values are passed, the method will compute predictions automatically:

In [ ]:
# Champion — Logistic regression model
vm_train_ds.assign_predictions(
    model=vm_log_model
)

vm_test_ds.assign_predictions(
    model=vm_log_model
)

# Challenger — Random forest classification model
vm_train_ds.assign_predictions(
    model=vm_rf_model
)

vm_test_ds.assign_predictions(
    model=vm_rf_model
)